In [1]:
# question_templates={
#     'missing_step' : 'Qual è lo step mancante nella seguente lista di step per la guida: {title}. Steps: {list_of_steps}',

#     'next_step' : 'Sapendo che il titolo della guida è {step}.Qual è il prossimo step in questa lista: {list_of_steps}',

#     'open_question' : 'Write the steps for this guide: {title}',

#     'rearrange' : 'Rearrange the steps for {title}: {list_of_steps}'
# }

# Questions
- remove a single step but print in the question all steps: step1 step2 ... step4 -> correct answer: step3
- predict the next step (minimum 4 steps before): step1 2 3 4 ... -> correct answer: step5
- Sequence Rearrangement. Provide the steps in a jumbled order and ask the user to arrange them in the correct   sequence.
Example:
Question: Rearrange the steps for “How to Tie a Shoelace”: Step 3, Step 1, Step 2, Step 4.
Correct Answer: Step 1, Step 2, Step 3, Step 4.
- open question write the steps for this guide
- difficulty: take the option step between the same guide (maximum difficulty) or from a different guide (easy)

In [2]:
import requests
from bs4 import BeautifulSoup,NavigableString
import random
from tqdm import tqdm
from multiprocessing import Pool, cpu_count


In [4]:
url='https://www.wikihow.com/Category:Gardening'
def get_all_pages(url,releted=True):
    base_url='https://www.wikihow.com'
    response = requests.get(url)
    pages=set()
    categories=set()
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        cat_sections=soup.find_all('div',class_='cat_section')
        for cat_section in cat_sections:
            if cat_section.get('id')=='cat_featured':
                #print('Feautured')
                articles=cat_section.find_all('div',class_='responsive_thumb')
                for article in articles:
                    url_a=article.find('a').get('href')
                    pages.add(url_a)
            if cat_section.get('id')=='cat_all':
                #print('ALL')
                articles=cat_section.find_all('div',class_='responsive_thumb')
                for article in articles:
                    url_a=article.find('a').get('href')
                    pages.add(url_a)
            if cat_section.get('id')=='cat_sub_categories' and releted:
                #print('SUBCAT')
                uls=cat_section.find_all('ul')
                for ul in uls:
                    for li in ul.find_all('li'):
                        #print(li)
                        for a in li.find_all('a',class_='cat_link'):
                            categories.add(base_url+a.get('href'))
        return pages,categories

In [5]:
urls,categories=get_all_pages(url,True)


In [6]:
urls2= set()
for category in categories:
    u,c = get_all_pages(category,False)
    urls2.update(u)
    if c:
        print(c)

In [7]:
urls

{'https://www.wikihow.com/Adjust-Hunter-Sprinklers',
 'https://www.wikihow.com/Apply-Liquid-Fertilizer',
 'https://www.wikihow.com/Apply-Mulch',
 'https://www.wikihow.com/Arrange-the-Inside-of-a-Greenhouse',
 'https://www.wikihow.com/Build-a-Lean-to-Shed',
 'https://www.wikihow.com/Build-a-Pergola',
 'https://www.wikihow.com/Clean-Garden-Stones',
 'https://www.wikihow.com/Clean-a-Slate-Hearth',
 'https://www.wikihow.com/Cut-Back-Irises-in-the-Fall',
 'https://www.wikihow.com/Cut-Back-Penstemon',
 'https://www.wikihow.com/Deal-with-Exposed-Tree-Roots',
 'https://www.wikihow.com/Determine-How-Much-Water-Plants-Need',
 'https://www.wikihow.com/Garden',
 'https://www.wikihow.com/Get-Rid-of-Morning-Glories',
 'https://www.wikihow.com/Grow-Bloodworms',
 'https://www.wikihow.com/Grow-Daikon',
 'https://www.wikihow.com/Grow-Pumpkins-Indoors',
 'https://www.wikihow.com/Grow-Sweet-Potato-Vine-Houseplant',
 'https://www.wikihow.com/Grow-Taro',
 'https://www.wikihow.com/Grow-Zucchini-in-Pots',
 'h

In [8]:
def process_soup(soup):
    ols=soup.find_all('ol',class_=lambda x: x and x.startswith('steps'))
    part=0
    page=[]
    title=url.split('/')[-1]
    divs=[]
    for ol in ols:
        section=[]
        part+=1
        step_num=0
        for li in ol.find_all('li', id=lambda x: x and x.startswith('step-id')):
            step={}
            step_num+=1
            img_tag = li.find('img')
            if img_tag and img_tag.get('data-src'):
                step_image = img_tag['data-src']
                step['image'] = step_image
            else:
                # Handle missing image or data-src attribute
                step['image'] = None
            step_div=li.find('div', class_='step')
            for element in step_div.children:
                if isinstance(element, NavigableString) and element.strip():
                    #non_tag_text += element.strip() + " "
                    text = step.get('text','')
                    text += element.strip() + " "
                    step['text'] = text
                # Check if element has the class 'whb'
                elif element.name and 'whb' in element.get("class", []):
                    step_title = step.get('title','')
                    step_title += element.get_text(strip=True)
                    step['title'] = step_title
                elif element.name == 'a' and 'external' in element.get("class", []):
                    #non_tag_text += element.text + " "
                    text = step.get('text','')
                    text += element.text + " "
                    step['text'] = text
                elif element.name == 'span' and 'frac' in element.get("class", []):
                    #non_tag_text += element.text + " "
                    text = step.get('text','')
                    fraction = element.text.replace('⁄','/')
                    text += fraction + " "
                    step['text'] = text
                elif element.name == 'span' and 'mwe-math-element' in element.get("class", []):
                    #non_tag_text += element.text + " "
                    text = step.get('text','')
                    fraction = element.text.replace('⁄','/')
                    text += fraction + " "
                    step['text'] = text
                elif element.name == 'a' and (not element.get("class") or 'intra-article' in element.get("class")):
                    #non_tag_text += element.text + " "
                    found=False
                    for child in element.children:
                        if child.name and 'whb' in child.get("class", []):
                            step_title = step.get('title','')
                            step_title += child.get_text(strip=True)
                            step['title'] = step_title 
                            found=True
                    if not found:
                        text = step.get('text','')
                        text += element.text + " "
                        step['text'] = text
                elif element.name == 'i' and not element.get("class"):
                    #non_tag_text += element.text + " " 
                    text = step.get('text','')
                    text += element.text + " "
                    step['text'] = text
                elif element.name == 'b' and not element.get("class"):
                    #non_tag_text += element.text + " "
                    found=False
                    for child in element.children:
                        if child.name and 'whb' in child.get("class", []):
                            step_title = step.get('title','')
                            step_title += child.get_text(strip=True)
                            step['title'] = step_title 
                            found=True
                    if not found:
                        text = step.get('text','')
                        text += element.text + " "
                        step['text'] = text
                elif element.name == 'div':
                    print(f'Div Found in part {part} step: {step_num} | Article: {url.split('/')[-1]}')
                    divs.append(element)
                elif element.name == 'ul':
                    ul_elem=[]
                    for li in element.find_all('li'):
                        plain_text = ''.join(text for text in li.contents if isinstance(text, str)).strip()
                        ul_elem.append(plain_text)
                    step_list = ''
                    for elem in ul_elem:
                        step_list += '\n- ' + elem
                    text = step.get('text','')
                    text += step_list + " "
                    step['text'] = text
                    #step['ul_lists']= step_list
                # Check if element is another tag
                elif element.name in ['script', 'sup','br','p','span','table']: #table could be processed
                    pass
                elif element.name:
                    print(f"Other tag ({element.name}) found in part {part} step: {step_num} | Article: {url.split('/')[-1]} |: {element}")
                else:
                    pass
            if not step.get('title') or len(step['title'])<5:
                print('Problem: ',title,step.get('title'),f'found in part {part} step: {step_num}')
            section.append(step)
        page.append(section)
    return {'title':title,'page' : page}, divs

In [9]:
#url = 'https://www.wikihow.com/Prune-a-Tree'
def get_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    else:
        print("Failed to retrieve the page.")

In [10]:
urls.update(urls2)

In [15]:
soups=[]
for url in tqdm(list(urls)[:100]):
    soup = get_page(url)
    soups.append(soup)

100%|██████████| 100/100 [00:26<00:00,  3.81it/s]


In [16]:
pages=[]
for url,soup in tqdm(zip(urls,soups),total=len(soups)):
    page,d=process_soup(soup)
    pages.append(page)

 57%|█████▋    | 57/100 [00:00<00:00, 86.40it/s]

Problem:  Meaning-of-the-Carnation Love found in part 1 step: 1


 93%|█████████▎| 93/100 [00:01<00:00, 85.21it/s]

Div Found in part 1 step: 1 | Article: Grow-Mushrooms
Div Found in part 1 step: 2 | Article: Grow-Mushrooms
Div Found in part 1 step: 2 | Article: Grow-Mushrooms


100%|██████████| 100/100 [00:01<00:00, 83.71it/s]


In [117]:
for page in pages[:100]:
    flat_steps = [step['title'] for section in page['page'] for step in section]
    page['steps'] = flat_steps
    print(page['title'],'\n')
    for n,step in enumerate(flat_steps):
            print(f'step_{n+1}',step)
    print('x'*20,'\n')

Graft-a-Fruit-Tree 

step_1 Wait until dormancy.
step_2 Choose young shoots.
step_3 Store in cool, moist conditions.
step_4 Trim the tip and base before use.
step_5 Choose a young, healthy tree.
step_6 Opt for a similar tree.
step_7 Graft in the spring.
step_8 Keep plenty of space between the trunk and the graft.
step_9 Cut the stock.
step_10 Slice a tongue into the stock branch.
step_11 Mirror the cut on the scion wood.
step_12 Match the cut wood together.
step_13 Tie and cover the graft.
step_14 Remove the wrapping once new growth develops.
step_15 Cut the stock branch.
step_16 Make a cleft into the cut.
step_17 Trim the scions.
step_18 Insert the scions into the cleft.
step_19 Cover the area with grafting compound.
step_20 Support the growing scions.
step_21 Gradually cut back the scions.
step_22 Cut into the bark of the stock.
step_23 Slice into the stock at an angle.
step_24 Make sloping cuts into the scion.
step_25 Join the scion and stock.
step_26 Nail the scions in place.
step_

In [90]:
flat_steps = [step['title'] for section in page['page'] for step in section]

In [232]:
question_templates = {

    'missing_step': 'What is the missing step in the following sequence for "{title}"?\n Steps: {list_of_steps}', # No option # Option with LLM

    'next_step': 'For "{title}", what comes next in this sequence? Steps: {list_of_steps}', # No option # Option with LLM

    'open_question': 'Write down all the steps for "{title}".',

    'rearrange': 'Put these steps in the correct order for "{title}": {list_of_steps}' # No LLM
}

def create_question(page, question_type):
    question = question_templates[question_type]
    list_of_steps=[]
    if question_type == 'missing_step':
        missing_step_num = random.randint(4,len(page['steps'])-1)
        list_of_steps = page['steps'][:]
        #correct_answer = page['steps'][missing_step_num]
        correct_answer = list_of_steps[missing_step_num]
        list_of_steps[missing_step_num] = "......"
    elif question_type == 'next_step':
        correct_answer = page['steps'][-1]
        list_of_steps = page['steps'].remove(-1)
    elif question_type == 'rearrange':
        correct_answer = page['steps']
        list_of_steps = random.shuffle(page['steps'])
    question=question.format(title=page['title'],list_of_steps=list_of_steps)
    options=[] # created using llm
    return {'question' : question, 'options' : options,'correct_answer' : correct_answer}


In [475]:
def create_question(page, question_type):
    question = question_templates[question_type]
    list_of_steps = []
    
    if question_type == 'missing_step':
        missing_step_num = random.randint(4, len(page['steps']) - 2)
        list_of_steps = page['steps'][:]
        correct_answer = list_of_steps[missing_step_num]
        list_of_steps[missing_step_num] = "............................."
    elif question_type == 'next_step':
        correct_answer = page['steps'][-1]
        list_of_steps = page['steps'][:-1]  # Remove the last step without modifying the original list
    elif question_type == 'rearrange':
        correct_answer = page['steps']
        list_of_steps = page['steps'][:]
        random.shuffle(list_of_steps)

    # Join the steps with newline characters
    formatted_steps = "\n".join(list_of_steps)
    question = question.format(title=page['title'], list_of_steps=formatted_steps)
    
    options = []  # Placeholder for options created using LLM
    return {'question': question, 'options': options, 'correct_answer': correct_answer}


In [233]:
question_templates['missing_step'].format(title='Stocazzo',list_of_steps=[])

'What is the missing step in the following sequence for "Stocazzo"?\n Steps: []'

In [756]:
question=create_question(page,'missing_step')
print(question['question'])
print('\n',question['correct_answer'])

What is the missing step in the following sequence for "Get-Rid-of-Snails"?
 Steps: Pick snails off by hand.
Usecopper tapeor wire to discourage snail activity.
Make a beer trap to capture the snails.
Spray the snails with vinegar.
Use a commercial snail bait.
Get a pet chicken that will eat the snails.
.............................
Call a professional if all else fails.
Water your soil in the morning to reduce egg laying.
Remove moist, decaying debris from your yard and garden.
Put up an abrasive barrier to deter snails.
Sprinkle used coffee grounds around your garden.
Add plants that naturally deter snails.
Slime trails
Snail eggs
Holes in leaves
Damaged seedlings
Chewed bark and flower buds
Quarantine plants for 2 weeks before putting them into your aquarium.
Treat plants in a bleach solution before adding them to your tank.
Remove snails by hand by cleaning the aquarium.
Introduce fish that eat snails.
Put a snail trap in your tank to catch and remove the snails.
Add fish-friendly 